<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/MusubiTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [検証中] Musubi-Tuner on Google Colab by AICU http://j.aicu.ai/MusubiTuner

サンプルの画像を使って Google Colabで Hunyuan Videoモデルに LoRAを学習させるスクリプトです。

- Kohyaさんによる原作のリポジトリ

https://github.com/kohya-ss/musubi-tuner/blob/main/README.ja.md

- AICUによる解説記事
Wan2.1の勢いがすごい!! Kohya氏によるLoRAツール「Musubi Tuner」が公開。Google Colabで動かしてみた！

https://note.com/aicu/n/nc37f872eae43


## Work in progress

- T4 ではVRAMがたりないのでキャッシュ生成で Out of Memory (OoM) します。L4 (22.5GB)で検証します。16.4GBぐらい必要なようです。
- キャプションを用意すべきですが、ファイル名をそのまま与えています。1girlなどにしたほうがいいかもしれません。
- 学習開始までたどり着きましたが、結果は検証していません。


In [1]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
!git clone https://github.com/kohya-ss/musubi-tuner.git
%cd musubi-tuner
!pip install -r requirements.txt

fatal: destination path 'musubi-tuner' already exists and is not an empty directory.
/content/musubi-tuner


↓ここでセッション再起動が入りますが、再度実行して継続してください。

In [3]:
!pip install ascii-magic matplotlib tensorboard

  Using cached ascii_magic-2.3.0-py3-none-any.whl.metadata (13 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached ascii_magic-2.3.0-py3-none-any.whl (733 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


In [4]:
!python -m pip install "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.7 MB/s eta 0:00:00


In [5]:
!mkdir HunyuanVideo
%cd HunyuanVideo

/content/musubi-tuner/HunyuanVideo


In [6]:
!huggingface-cli download tencent/HunyuanVideo --local-dir ./ckpts

Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]Downloading 'hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt' to 'ckpts/.cache/huggingface/download/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt.99fbade8af80b5233194e4e8afcaa3ca68a412fce6403a440f962a432fc18a9c.incomplete'

mp_rank_00_model_states_fp8.pt:   0% 0.00/13.2G [00:00<?, ?B/s]

mp_rank_00_model_states_fp8_map.pt: 100% 104k/104k [00:00<00:00, 24.8MB/s]
Download complete. Moving file to ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8_map.pt


mp_rank_00_model_states.pt:   0% 0.00/25.6G [00:00<?, ?B/s]
mp_rank_00_model_states_fp8.pt:   0% 21.0M/13.2G [00:00<01:55, 114MB/s]Downloading 'hunyuan-video-t2v-720p/vae/config.json' to 'ckpts/.cache/huggingface/download/hunyuan-video-t2v-720p/vae/config.json.579df939e5ffcacd57d9eecc0ced31a76b0ecee7.incomplete'


mp_rank_00_model_states.pt:   0% 21.0M/25.6G [00:00<03:10, 134MB/s]


config.json: 100% 39.0/39.0 [00:00<00:00, 305kB/s]
D

データセットの設定を行うためのTOMLファイルを作成してください。

https://github.com/kohya-ss/musubi-tuner/blob/main/dataset/dataset_config.md

画像データセットと動画データセットがサポートされています。設定ファイルには、画像または動画データセットを複数含めることができます。キャプションテキストファイルまたはメタデータJSONLファイルを使用できます。

キャッシュディレクトリは、各データセットごとに異なるディレクトリである必要があります。

In [11]:
#仮のデータとして https://huggingface.co/AICU/SDXL-LoRA/blob/main/9shoku0219.zip をダウンロードして展開する
%cd /content/musubi-tuner

!wget -c https://huggingface.co/AICU/SDXL-LoRA/resolve/main/9shoku0219.zip -P ./
!unzip 9shoku0219.zip


/content/musubi-tuner
--2025-03-08 22:17:10--  https://huggingface.co/AICU/SDXL-LoRA/resolve/main/9shoku0219.zip
Resolving huggingface.co (huggingface.co)... 3.165.102.58, 3.165.102.6, 3.165.102.128, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.58|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/15/99/15991d153f50a7b2277059791040f32902fc56f04850c101d5fd6ddcf9c1df10/7a0fc3c64061ed228293a381017407bd260ed67742cc6c903c02b223a99ea221?response-content-disposition=inline%3B+filename*%3DUTF-8%27%279shoku0219.zip%3B+filename%3D%229shoku0219.zip%22%3B&response-content-type=application%2Fzip&Expires=1741475830&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTQ3NTgzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzE1Lzk5LzE1OTkxZDE1M2Y1MGE3YjIyNzcwNTk3OTEwNDBmMzI5MDJmYzU2ZjA0ODUwYzEwMWQ1ZmQ2ZGRjZjljMWRmMTAvN2EwZmMzYzY0MDYxZWQyMjgyOTNhMzgxMDE3NDA3YmQyNjBlZDY3NzQy

データセットの設定を行うためのTOMLファイルを作成してください。
https://github.com/kohya-ss/musubi-tuner/blob/main/dataset/dataset_config.md
画像データセットと動画データセットがサポートされています。設定ファイルには、画像または動画データセットを複数含めることができます。キャプションテキストファイルまたはメタデータJSONLファイルを使用できます。

キャッシュディレクトリは、各データセットごとに異なるディレクトリである必要があります。

### Sample for Image Dataset with Caption Text Files

```
# resolution, caption_extension, batch_size, num_repeats, enable_bucket, bucket_no_upscale should be set in either general or datasets
# otherwise, the default values will be used for each item

# general configurations
[general]
resolution = [960, 544]
caption_extension = ".txt"
batch_size = 1
enable_bucket = true
bucket_no_upscale = false

[[datasets]]
image_directory = "/path/to/image_dir"
cache_directory = "/path/to/cache_directory"
num_repeats = 1 # optional, default is 1. Number of times to repeat the dataset. Useful to balance the multiple datasets with different sizes.

# other datasets can be added here. each dataset can have different configurations
```


cache_directory はオプションです。デフォルトは画像ディレクトリと同じディレクトリに設定されます。ただし、異なるデータセット間でキャッシュファイルが共有されるのを防ぐために、明示的に別のキャッシュディレクトリを設定することをお勧めします。

num_repeats はオプションで、デフォルトは 1 です（繰り返しなし）。画像（や動画）を、その回数だけ単純に繰り返してデータセットを拡張します。たとえばnum_repeats = 2としたとき、画像20枚のデータセットなら、各画像が2枚ずつ（同一のキャプションで）計40枚存在した場合と同じになります。異なるデータ数のデータセット間でバランスを取るために使用可能です。

resolution, caption_extension, batch_size, num_repeats, enable_bucket, bucket_no_upscale は general または datasets のどちらかに設定してください。省略時は各項目のデフォルト値が使用されます。

[[datasets]]以下を追加することで、他のデータセットを追加できます。各データセットには異なる設定を持てます。

### Sample for Image Dataset with Metadata JSONL File

```
# resolution, batch_size, num_repeats, enable_bucket, bucket_no_upscale should be set in either general or datasets
# caption_extension is not required for metadata jsonl file
# cache_directory is required for each dataset with metadata jsonl file

# general configurations
[general]
resolution = [960, 544]
batch_size = 1
enable_bucket = true
bucket_no_upscale = false

[[datasets]]
image_jsonl_file = "/path/to/metadata.jsonl"
cache_directory = "/path/to/cache_directory" # required for metadata jsonl file
num_repeats = 1 # optional, default is 1. Same as above.

# other datasets can be added here. each dataset can have different configurations
```



In [12]:
# JSONL Fileを生成

import json
import os

# Create the metadata.jsonl file
metadata_file = "metadata.jsonl"
image_dir = "/content/musubi-tuner/9shoku0219/02189shoku"  # Path to your image directory

with open(metadata_file, "w") as f:
    for filename in os.listdir(image_dir):
        if filename.endswith((".png", ".jpg", ".jpeg")):  # Adjust extensions as needed
            filepath = os.path.join(image_dir, filename)
            data = {
                "image_path": filepath,
                "caption": filename, # Replace with actual captions if available
            }
            json.dump(data, f)
            f.write("\n")


# Create the config.toml file
config_toml = """
# resolution, batch_size, num_repeats, enable_bucket, bucket_no_upscale should be set in either general or datasets
# caption_extension is not required for metadata jsonl file
# cache_directory is required for each dataset with metadata jsonl file
# general configurations
[general]
resolution = [1024, 1024]
batch_size = 1
enable_bucket = true
bucket_no_upscale = false
[[datasets]]
image_jsonl_file = "/content/musubi-tuner/metadata.jsonl"
cache_directory = "/content/musubi-tuner/cache" # required for metadata jsonl file
num_repeats = 1 # optional, default is 1. Same as above.
# other datasets can be added here. each dataset can have different configurations
"""

with open("config.toml", "w") as f:
  f.write(config_toml)

!mkdir /content/musubi-tuner/cache


In [13]:
%cd /content/musubi-tuner

# latentの事前キャッシュ
!python cache_latents.py --dataset_config /content/musubi-tuner/config.toml --vae /content/musubi-tuner/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/vae/pytorch_model.pt --vae_chunk_size 32 --vae_tiling


/content/musubi-tuner
INFO:__main__:Load dataset config from /content/musubi-tuner/config.toml
INFO:dataset.image_video_dataset:load image jsonl from /content/musubi-tuner/metadata.jsonl
INFO:dataset.image_video_dataset:loaded 30 images
INFO:dataset.config_utils:[Dataset 0]
  is_image_dataset: True
  resolution: (1024, 1024)
  batch_size: 1
  num_repeats: 1
  caption_extension: "None"
  enable_bucket: True
  bucket_no_upscale: False
  cache_directory: "/content/musubi-tuner/cache"
  debug_dataset: False
    image_directory: "None"
    image_jsonl_file: "/content/musubi-tuner/metadata.jsonl"


INFO:hunyuan_model.vae:Loading 3D VAE model (884-16c-hy) from: /content/musubi-tuner/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/vae/pytorch_model.pt
INFO:hunyuan_model.vae:VAE to dtype: torch.float16
INFO:__main__:Loaded VAE: FrozenDict([('in_channels', 3), ('out_channels', 3), ('down_block_types', ['DownEncoderBlockCausal3D', 'DownEncoderBlockCausal3D', 'DownEncoderBlockCausal3D', 'DownEncoderBloc

## Text EncoderにComfyUI提供のモデルを使う
こちらの方法の方がより簡単です。DiTとVAEのモデルはHumyuanVideoのものを使用します。

https://huggingface.co/tencent/HunyuanVideo/tree/main/hunyuan-video-t2v-720p/transformers から、mp_rank_00_model_states.pt をダウンロードし、任意のディレクトリに配置します。

（同じページにfp8のモデルもありますが、未検証です。）

--fp8_baseを指定して学習する場合は、mp_rank_00_model_states.ptの代わりに、こちらのmp_rank_00_model_states_fp8.safetensorsを使用可能です。（このファイルは非公式のもので、重みを単純にfloat8_e4m3fnに変換したものです。）

また、https://huggingface.co/tencent/HunyuanVideo/tree/main/hunyuan-video-t2v-720p/vae から、pytorch_model.pt をダウンロードし、任意のディレクトリに配置します。

Text EncoderにはComfyUI提供のモデルを使用させていただきます。ComyUIのページを参考に、https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/tree/main/split_files/text_encoders から、llava_llama3_fp16.safetensors （Text Encoder 1、LLM）と、clip_l.safetensors （Text Encoder 2、CLIP）をダウンロードし、任意のディレクトリに配置します。

（同じページにfp8のLLMモデルもありますが、動作未検証です。）

In [14]:
# Text Encoder出力の事前キャッシュ
%cd /content/musubi-tuner
!mkdir Comfy-Org
%cd Comfy-Org/
!wget -c https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/clip_vision/llava_llama3_vision.safetensors -P ./
!wget -c https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors -P ./
!wget -c https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp16.safetensors -P ./


/content/musubi-tuner
/content/musubi-tuner/Comfy-Org
--2025-03-08 22:19:57--  https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/clip_vision/llava_llama3_vision.safetensors
Resolving huggingface.co (huggingface.co)... 3.165.102.128, 3.165.102.6, 3.165.102.58, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/21/1b/211b54f9371a9d6b32e56501554c4eb1a26c02c6e75ca8a64bbbe724b52c67ee/7d0f89bf7860815f3a994b9bdae8ebe3a29c161825d03ca9262cb13b0c973aa6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llava_llama3_vision.safetensors%3B+filename%3D%22llava_llama3_vision.safetensors%22%3B&Expires=1741475997&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTQ3NTk5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzIxLzFiLzIxMWI1NGY5MzcxYTlkNmIzMmU1NjUwMTU1NGM0ZWIxYTI2YzAyYzZlN

In [15]:
%cd /content/musubi-tuner

!python cache_text_encoder_outputs.py --dataset_config /content/musubi-tuner/config.toml --text_encoder1 /content/musubi-tuner/Comfy-Org/llava_llama3_fp16.safetensors --text_encoder2 /content/musubi-tuner/Comfy-Org/clip_l.safetensors --batch_size 16



/content/musubi-tuner
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2025-03-08 22:21:40.604342: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 22:21:41.213604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741472501.493833   10405 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741472501.565282   10405 cuda_blas.

In [ ]:
%cd /content/musubi-tuner
!accelerate launch --num_cpu_threads_per_process 1 --mixed_precision bf16 hv_train_network.py --dit /content/musubi-tuner/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states.pt --dataset_config /content/musubi-tuner/config.toml --sdpa --mixed_precision bf16 --fp8_base --optimizer_type adamw8bit --learning_rate 2e-4 --gradient_checkpointing --max_data_loader_n_workers 2 --persistent_data_loader_workers --network_module networks.lora --network_dim 32 --timestep_sampling shift --discrete_flow_shift 7.0 --max_train_epochs 16 --save_every_n_epochs 1 --seed 42 --output_dir /content/musubi-tuner/9shoku0219 --output_name 9shokuWan2LoRA

/content/musubi-tuner
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Trying to import sageattention
Failed to import sageattention
2025-03-08 22:28:06.111655: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 22:28:06.128562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741472886.148240   12127 cuda_dnn.cc:8310] Unable to register cuDN